In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

In [2]:
bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=True)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

Using downloaded and verified file: ./mnist_data/MNIST\raw\train-images-idx3-ubyte.gz
Extracting ./mnist_data/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist_data/MNIST\raw
Using downloaded and verified file: ./mnist_data/MNIST\raw\train-labels-idx1-ubyte.gz
Extracting ./mnist_data/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist_data/MNIST\raw
Using downloaded and verified file: ./mnist_data/MNIST\raw\t10k-images-idx3-ubyte.gz
Extracting ./mnist_data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST\raw
Using downloaded and verified file: ./mnist_data/MNIST\raw\t10k-labels-idx1-ubyte.gz
Extracting ./mnist_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST\raw
Processing...
Done!


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

In [4]:
# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [5]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [6]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [7]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))


In [8]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


In [9]:
for epoch in range(1, 51):
    train(epoch)
    test()

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Loss: 545.309102
Train Epoch: 1 [10000/60000 (17%)]	Loss: 193.969551
Train Epoch: 1 [20000/60000 (33%)]	Loss: 177.935410
Train Epoch: 1 [30000/60000 (50%)]	Loss: 174.585098
Train Epoch: 1 [40000/60000 (67%)]	Loss: 166.035371
Train Epoch: 1 [50000/60000 (83%)]	Loss: 157.933281
====> Epoch: 1 Average loss: 180.4065
====> Test set loss: 162.2448
Train Epoch: 2 [0/60000 (0%)]	Loss: 157.433564
Train Epoch: 2 [10000/60000 (17%)]	Loss: 158.580830
Train Epoch: 2 [20000/60000 (33%)]	Loss: 156.189404
Train Epoch: 2 [30000/60000 (50%)]	Loss: 159.358633
Train Epoch: 2 [40000/60000 (67%)]	Loss: 149.771670
Train Epoch: 2 [50000/60000 (83%)]	Loss: 156.987383
====> Epoch: 2 Average loss: 157.9970
====> Test set loss: 154.3614
Train Epoch: 3 [0/60000 (0%)]	Loss: 157.602139
Train Epoch: 3 [10000/60000 (17%)]	Loss: 159.736758
Train Epoch: 3 [20000/60000 (33%)]	Loss: 163.745020
Train Epoch: 3 [30000/60000 (50%)]	Loss: 149.100176
Train Epoch: 3 [40000/60000 (67%)]	Loss: 154.81

Train Epoch: 22 [50000/60000 (83%)]	Loss: 133.307412
====> Epoch: 22 Average loss: 139.5319
====> Test set loss: 141.1050
Train Epoch: 23 [0/60000 (0%)]	Loss: 134.999746
Train Epoch: 23 [10000/60000 (17%)]	Loss: 140.988975
Train Epoch: 23 [20000/60000 (33%)]	Loss: 143.493418
Train Epoch: 23 [30000/60000 (50%)]	Loss: 138.890068
Train Epoch: 23 [40000/60000 (67%)]	Loss: 141.706797
Train Epoch: 23 [50000/60000 (83%)]	Loss: 140.656807
====> Epoch: 23 Average loss: 139.0740
====> Test set loss: 140.4942
Train Epoch: 24 [0/60000 (0%)]	Loss: 142.417148
Train Epoch: 24 [10000/60000 (17%)]	Loss: 138.434336
Train Epoch: 24 [20000/60000 (33%)]	Loss: 142.098174
Train Epoch: 24 [30000/60000 (50%)]	Loss: 138.564482
Train Epoch: 24 [40000/60000 (67%)]	Loss: 138.518301
Train Epoch: 24 [50000/60000 (83%)]	Loss: 137.757422
====> Epoch: 24 Average loss: 138.8497
====> Test set loss: 140.8278
Train Epoch: 25 [0/60000 (0%)]	Loss: 142.967783
Train Epoch: 25 [10000/60000 (17%)]	Loss: 138.291426
Train Epoch: 

Train Epoch: 44 [20000/60000 (33%)]	Loss: 138.056094
Train Epoch: 44 [30000/60000 (50%)]	Loss: 140.111357
Train Epoch: 44 [40000/60000 (67%)]	Loss: 135.638438
Train Epoch: 44 [50000/60000 (83%)]	Loss: 145.357812
====> Epoch: 44 Average loss: 136.1365
====> Test set loss: 138.4053
Train Epoch: 45 [0/60000 (0%)]	Loss: 142.933994
Train Epoch: 45 [10000/60000 (17%)]	Loss: 129.790059
Train Epoch: 45 [20000/60000 (33%)]	Loss: 130.571875
Train Epoch: 45 [30000/60000 (50%)]	Loss: 136.864385
Train Epoch: 45 [40000/60000 (67%)]	Loss: 135.519570
Train Epoch: 45 [50000/60000 (83%)]	Loss: 139.282939
====> Epoch: 45 Average loss: 136.2117
====> Test set loss: 138.4775
Train Epoch: 46 [0/60000 (0%)]	Loss: 139.703486
Train Epoch: 46 [10000/60000 (17%)]	Loss: 135.144844
Train Epoch: 46 [20000/60000 (33%)]	Loss: 130.840205
Train Epoch: 46 [30000/60000 (50%)]	Loss: 139.607119
Train Epoch: 46 [40000/60000 (67%)]	Loss: 134.026064
Train Epoch: 46 [50000/60000 (83%)]	Loss: 137.949404
====> Epoch: 46 Average 

In [10]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = vae.decoder(z).cuda()
    
    save_image(sample.view(64, 1, 28, 28), 'sample' + '.png')